In [7]:
from ensemble_boxes import nms, soft_nms, non_maximum_weighted, weighted_boxes_fusion

import os
import shutil
import pandas as pd
import numpy as np

## CSV file load.

In [2]:
save_paths = [
    "submission_universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco",
    "submission_htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco_no_seg_with_tta",
    "submission_htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco_no_seg",
    "submission_vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco_with_tta",
    "submission_vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco",
    "submission_cascade_mask_rcnn_swin_base_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco_with_tta",
    "submission_cascade_mask_rcnn_swin_base_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco"
]
lb_scores = [7, 6, 1, 4, 3, 5, 2]

data_dfs = [pd.read_csv(os.path.join("save_dir", "Weighted_Boxes_Fusion", f"{save_path}.csv")) for save_path in save_paths]

## Helper function define.

In [3]:
def get_values(predicted):
    predicted = predicted.split(" ")
    
    labels = []
    scores = []
    boxes = []
    
    for i in range(0, len(predicted) - 1, 6):
        label = int(predicted[i])
        score = float(predicted[i + 1])
        box = list(map(lambda x: float(x) / 512, predicted[i + 2:i + 6]))
        
        labels.append(label)
        scores.append(score)
        boxes.append(box)
        
    return boxes, scores, labels

## Weighted boxes fusion.

In [4]:
iou_thr = 0.5
skip_box_thr = 0.0001

In [5]:
wbf_results = []

for idx in range(len(data_dfs[0])):
    boxes_ = []
    scores_ = []
    labels_ = []
    for df_idx in range(len(data_dfs)):
        predicted = data_dfs[df_idx].iloc[idx, 0]
        box, score, label = get_values(predicted)
        boxes_.append(box)
        scores_.append(score)
        labels_.append(label)
        
    wbf_box, wbf_score, wbf_label = weighted_boxes_fusion(boxes_, scores_, labels_, weights=lb_scores, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    wbf_result = [[int(wbf_label_)] + [wbf_score_] + list(map(lambda x: x * 512, wbf_box_)) for wbf_box_, wbf_score_, wbf_label_ in zip(wbf_box, wbf_score, wbf_label)]
    wbf_result = sorted(wbf_result, key=lambda x: x[1], reverse=True)
    wbf_result = sorted(wbf_result, key=lambda x: x[0])
    
    wbf_results.append(wbf_result)

/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.98146552734375, 1.0, 0.9999171484375, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [2.8412580078125e-05, 1.0, 0.00635201953125, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [7.08193046875e-05, 1.0, 0.015839076171875, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 1.0, 0.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped

## Save to CSV file.

In [6]:
image_names = data_dfs[0].iloc[:, 1]

prediction_strings = []
file_names = []

for wbf_result, image_name in zip(wbf_results, image_names):
    prediction_string = ""
    for wbf_result_ in wbf_result:
        label = str(wbf_result_[0])
        score = str(wbf_result_[1])
        box = " ".join(list(map(lambda x: str(x), wbf_result_[2:])))
        prediction_string += label + " " + score + " " + box + " "
    
    prediction_strings.append(prediction_string)
    file_names.append(image_name)
    
submission = pd.DataFrame()
submission["PredictionString"] = prediction_strings
submission["image_id"] = file_names
submission.to_csv(os.path.join("save_dir", f"submission_wbf_ensemble.csv"), index=None)
submission.head()

PredictionString              image_id
0  0 0.09352351725101471 454.3958435058594 237.53...  batch_01_vt/0021.jpg
1  0 0.09406907856464386 271.91241455078125 242.1...  batch_01_vt/0028.jpg
2  0 0.10284692049026489 235.69442749023438 460.8...  batch_01_vt/0031.jpg
3  0 0.14967824518680573 297.0133972167969 158.77...  batch_01_vt/0032.jpg
4  0 0.15966905653476715 113.53695678710938 252.9...  batch_01_vt/0070.jpg